# 🎗️ Breast Cancer Classification — ANN (Keras)
A clean, reproducible notebook that trains an **Artificial Neural Network (ANN)** to classify **benign vs malignant** tumors on the **Breast Cancer Wisconsin (Diagnostic)** dataset.

> This notebook mirrors the structure shown in your PDF and is ready to run inside your repo.

## 1) Setup & Imports

In [ ]:

# Core
import numpy as np
import pandas as pd

# Scikit-learn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Viz
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


## 2) Load & Explore Data

In [ ]:

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")  # 0=malignant, 1=benign

print("Shape:", X.shape, y.shape)
X.head()


## 3) Train/Test Split & Scaling (no leakage)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled[:3]


## 4) Build ANN (Keras Sequential)

In [ ]:

input_dim = X_train_scaled.shape[1]

model = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


## 5) Train

In [ ]:

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=0
)

# Plot learning curves
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(history.history['accuracy'], label='train acc')
ax.plot(history.history['val_accuracy'], label='val acc')
ax.set_xlabel('Epoch'); ax.set_ylabel('Accuracy'); ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.legend(); plt.show()

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(history.history['loss'], label='train loss')
ax.plot(history.history['val_loss'], label='val loss')
ax.set_xlabel('Epoch'); ax.set_ylabel('Loss'); ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.legend(); plt.show()


## 6) Evaluate

In [ ]:

# Accuracy
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", round(acc, 4))

# Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=data.target_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# ROC-AUC
auc = roc_auc_score(y_test, y_pred_prob)
print("\nROC-AUC:", round(auc, 4))

# Plot ROC
fpr, tpr, thr = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"ROC-AUC={auc:.3f}")
plt.plot([0,1],[0,1],'--', linewidth=1)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curve'); plt.legend(); plt.show()


## 7) Inference Helper (single sample)

In [ ]:

def predict_single(sample: np.ndarray):
    sample = sample.reshape(1, -1)
    sample_scaled = scaler.transform(sample)
    prob = float(model.predict(sample_scaled)[0][0])
    return prob, int(prob >= 0.5)

# Example with the first test row
p, label = predict_single(X_test.iloc[0].values)
print("Pred prob (benign):", round(p, 4), "→ Pred label:", label, "(0=malignant, 1=benign)")


## 8) Save Artifacts (optional)

In [ ]:

# Save model and scaler if needed
# model.save('models/ann_breast_cancer.h5')
# import joblib; joblib.dump(scaler, 'models/scaler.joblib')
print("Artifacts saving is commented out by default.")
